In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [3]:
df = pd.read_csv("final_model.csv")
df

,make_model,body_type,price,km,hp,Gearing Type,Displacement_cc,Fuel,Age
0,Audi A1,Sedans,15770,56013,88.507458,Automatic,1422.0,Diesel,3.0
1,Audi A1,Sedans,14500,80000,189.084115,Automatic,1798.0,Benzine,2.0
2,Audi A1,Sedans,14640,83450,113.986878,Automatic,1598.0,Diesel,3.0
3,Audi A1,Sedans,14500,73000,88.507458,Automatic,1422.0,Diesel,3.0
4,Audi A1,Sedans,16790,16200,88.507458,Automatic,1422.0,Diesel,3.0
...,...,...,...,...,...,...,...,...,...
15909,Renault Espace,Van,39950,0,197.130247,Automatic,1997.0,Diesel,0.0
15910,Renault Espace,Van,39885,9900,221.268645,Automatic,1798.0,Benzine,0.0
15911,Renault Espace,Van,39875,15,195.789225,Automatic,1997.0,Diesel,0.0
15912,Renault Espace,Van,39700,10,197.130247,Automatic,1997.0,Diesel,0.0


In [4]:
from sklearn.preprocessing import OrdinalEncoder

In [5]:
X = df.drop("price", axis=1)
y = df.price

In [6]:
cat = X.select_dtypes("object").columns
cat

Index(['make_model', 'body_type', 'Gearing Type', 'Fuel'], dtype='object')

In [7]:
enc = OrdinalEncoder()
X[cat] = enc.fit_transform(X[cat])
X.head()

,make_model,body_type,km,hp,Gearing Type,Displacement_cc,Fuel,Age
0,0.0,4.0,56013,88.507458,0.0,1422.0,1.0,3.0
1,0.0,4.0,80000,189.084115,0.0,1798.0,0.0,2.0
2,0.0,4.0,83450,113.986878,0.0,1598.0,1.0,3.0
3,0.0,4.0,73000,88.507458,0.0,1422.0,1.0,3.0
4,0.0,4.0,16200,88.507458,0.0,1422.0,1.0,3.0


In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=24)
print("Train features shape : ", X_train.shape)
print("Train target shape   : ", y_train.shape)
print("Test features shape  : ", X_test.shape)
print("Test target shape    : ", y_test.shape)

Train features shape :  (12731, 8)
Train target shape   :  (12731,)
Test features shape  :  (3183, 8)
Test target shape    :  (3183,)


In [9]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
def train_val(model, X_train, y_train, X_test, y_test):
    
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    scores = {"train": {"R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),                          
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},
    
    "test": {"R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}}
    
    return pd.DataFrame(scores)

In [10]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(random_state=24, objective="reg:squarederror") #objective="reg:squarederror"
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=24,
             reg_alpha=0, reg_lambda=1, ...)

In [11]:
train_val(xgb_model, X_train, y_train, X_test, y_test)

,train,test
R2,9.661326e-01,9.299014e-01
mae,8.824345e+02,1.162608e+03
mse,1.836273e+06,3.892817e+06
rmse,1.355092e+03,1.973022e+03


In [ ]:
# model scoru icin r2ye baktik 
# train ve test r2 ikisi de yuksek, bizim icin iyi, grid searche gerek yok 

In [12]:
import pickle
pickle.dump(xgb_model, open('autoscout.pkl','wb'))
pickle.dump(enc, open('autoscout_encoder.pkl','wb'))
# model ve encoder dump etmis olduk 
# encoder kullanacagimiz icin onu da dump ettik 